# US EPA Air Quality System API Example
This example illustrates how to request data from the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. This is a historical API and does not provide real-time air quality data. The [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

This notebook works systematically through example calls, requesting an API key, using 'list' to get various IDs and parameter values, and using 'daily summary' to get summary data that meets specific condistions. The notebook contains example function definitions that could be reused in other code. In general, the notebook explains each step along the way, referring back to possible output. Some of the explanations are tailored to the specific example requests of the API. Changing values to explore the results of the API is probably useful, but that will result in some explanations being out of sync with the outputs.

The US EPA was created in the early 1970's. The EPA reports that they only started broad based monitoring with standardized quality assurance procedures in the 1980's. Many counties will have data starting somewhere between 1983 and 1988. However, some counties still do not have any air quality monitoring stations. The API helps resolve this by providing calls to search for monitoring stations and data using either station ids, or a county designation or a geographic bounding box. This example code provides examples of the county based and bounding box based API calls. Some [additional information on the Air Quality System can be found in the EPA FAQ](https://www.epa.gov/outdoor-air-quality-data/frequent-questions-about-airdata) on the system.

The end goal of this example is to get to some values that we might use for the Air Quality Index or AQI. You might see this reported on the news, most often around smog, but more frequently with regard to smoke. The AQI index is meant to tell us something about how healthy or clean the air is on any day. The AQI is actually a somewhat complext measure. When I started this example I looked up [how to calculate the AQI](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf) so that I would know roughly what goes into that value.


## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.2 - August 16, 2024

The following pieces of code were used from Dr. David W. McDonald's Python Notebook File `epa_air_quality_history_example.ipynb`: 
1. API Request Template
2. `request_signup`
3. `request_list_info`
4. `request_daily_summary`


The rest of the code lies under the standard MIT license.


In [ ]:
#
# External Libraries
#

import pandas as pd
import numpy as np

#### Standard Python Libraries Required

In [39]:
# 
#    These are standard python modules
#
#import json, time, urllib.parse
import json, time
#
#    The 'requests' module is a distribution module for making web requests. If you do not have it already, you'll need to install it
import requests

This below code defines constants and templates for interacting with the Air Quality System (AQS) API managed by the EPA. The `API_REQUEST_URL` serves as the base URL for all API requests. Various endpoint actions are provided, including `signup` to request an API key, `list` actions to retrieve data classes, parameters, and sites, and `monitor` actions to gather monitoring station data based on specific criteria, such as county or geographical bounding box. It also includes `summary` actions to access daily data summaries in county or bounding-box areas. 

In this project, we will be proceeding with using the daily data summaries approach.

An API request template (`AQS_REQUEST_TEMPLATE`) is prepared with placeholder fields for commonly used parameters like email, API key, geographic and date ranges, and parameters for data requests. Additionally, a throttle setting (`API_THROTTLE_WAIT`) is established to maintain a respectful rate limit, assuming an estimated latency for each request. This setup provides a structured approach for handling varied air quality data requests efficiently.

In [40]:
#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are some of the 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
# 
AQS_REQUEST_TEMPLATE = {
    "email":      "",     
    "key":        "",      
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}



## Example 1. Making a sign-up request

Before you can use the API you need to request a key. You will use an email address to make the request. The EPA then sends a confirmation email link and a 'key' that you use for all other requests.

You only need to sign-up once, unless you want to invalidate your current key (by getting a new key) or you lose your key.


In [41]:
#
#    This implements the sign-up request. The parameters are standardized so that this function definition matches
#    all of the others. However, the easiest way to call this is to simply call this function with your preferred
#    email address.
#
def request_signup(email_address = None,
                   endpoint_url = API_REQUEST_URL, 
                   endpoint_action = API_ACTION_SIGNUP, 
                   request_template = AQS_REQUEST_TEMPLATE,
                   headers = None):
    
    # Make sure we have a string - if you don't have access to this email addres, things might go badly for you
    if email_address:
        request_template['email'] = email_address        
    
    if not request_template['email']: 
        raise Exception("Must supply an email address to call 'request_signup()'")

    if '@' not in request_template['email']: 
        raise Exception(f"Must supply an email address to call 'request_signup()'. The string '{request_template['email']}' does not look like an email address.")

    # Compose the signup url - create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



A SIGNUP request is only to be done once, to request a key. A key is sent to that email address and needs to be confirmed with a click through.

This below code should probably be commented out after you've made your key request to make sure you don't accidentally make a new sign-up request

In [42]:

#    A SIGNUP request is only to be done once, to request a key. A key is sent to that email address and needs to be confirmed with a click through
#    This code should probably be commented out after you've made your key request to make sure you don't accidentally make a new sign-up request

# print("Requesting SIGNUP ...")
# USERNAME = "needula@uw.edu"
# response = request_signup(USERNAME)
# print(json.dumps(response,indent=4))


A response to the signup request might look something like this.


    Requesting SIGNUP ...
    {
        "Header": [
            {
                "status": "Success",
                "request_time": "2023-08-07T17:03:27-04:00",
                "url": "https://aqs.epa.gov/data/api/signup?email=dwmc@uw.edu"
            }
        ],
        "Data": [
            "You should receive a registration confirmation email with a link for confirming your email shortly."
        ]
    }



**Note:** The code below relies on a user module called apikeys that is a key manager. You don't need that to make this code work. You can use the email you used for your sign-up request and the key that you got in email. Assign your email address to "USERNAME" and your key to "APIKEY" as constants and the remaining cells in the notbook should work.

I've used my API key and removed it from the code below to avoid exposing sensitive this information. The `APIKEY` filed below can be replaced by your own key after doing the sign up request.

In [43]:
#
# Credentials to use API
#

USERNAME = "needula@uw.edu"
APIKEY = ""

## Example 2. Making a list request
Once you have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. The one problem here is that some of the responses rely on jargon or terms-of-art. That is, one needs to know a bit about the way atmospheric sciece works to understand some of the terms.

This code defines `request_list_info`, a function that retrieves various metadata lists from the AQS API, such as groups or parameter class descriptors. This information is essential for identifying specific air quality measurement codes, allowing users to later make targeted requests using proprietary 5-digit codes. The function takes parameters for `email_address` and `key`, which are necessary for accessing the API, and assigns them to a `request_template` if provided. If either is missing, it raises an exception. The function then constructs a request URL by combining the API’s base URL and action for list requests, formatted with the template values. Before sending the request, a throttle delay is applied to avoid surpassing API rate limits. The function attempts to make a `GET` request, parses the response as JSON, and handles any errors by printing the exception and returning `None`. This setup simplifies retrieving API parameter descriptions and codes required for more specific air quality data requests.

In [44]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors 
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clusters of sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from that specific sensor.

In [45]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

We're interested in getting to something that might be the Air Quality Index (AQI). You see this reported on the news - often around smog values, but also when there is smoke in the sky. The AQI is a complex measure of different gasses and of the particles in the air (dust, dirt, ash ...).

From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.


Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group). The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS". We'll use that to make another list request.

In [47]:
AQI_PARAM_CLASS = "AQI POLLUTANTS"

#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We should now have (above) a response containing a set of sensor ID numbers. The list should include the sensor numbers as well as a description or name for each sensor. 

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up.

What we did below was to break the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [48]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#   
#

Air quality monitoring stations are located all over the US at different locations. We will need some sample locations to experiment with different locations to see what kinds of values come back from different sensor requests.

This list includes the [FIPS](https://www.census.gov/library/reference/code-lists/ansi.html) number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.

We are interested in exploring the AQI data for the city Centennial, CO.

The below fields were filled by doing a simple Google Search to find the county, and the fips. The latitude and longitude were taken from the original source - the GeoHack website.

In [49]:
CITY_LOCATIONS = {
    'centennial' :     {'city'   : 'Centennial',
                    'county' : 'Arapahoe',
                    'state'  : 'Colorado',
                    'fips'   : '08005',
                    'latlon' : [39.59, -104.87]  
                         }
}

For this city, we are using the EPA to list their monitoring stations by county. 

This list request should give us a list of all the monitoring stations in the county specified by the given city selected from the CITY_LOCATIONS dictionary

For my city, Centennial, the API request returned 14 monitoring stations. I deemed that these 14 stations would collect enough data for the AQI. Hence, I picked this approach over the bounding box approach.

In [50]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['centennial']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['centennial']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "0001",
        "value_represented": "CHERRY CREEK DAM"
    },
    {
        "code": "0002",
        "value_represented": "HIGHLAND RESERVOIR"
    },
    {
        "code": "0003",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "0005",
        "value_represented": "Arapahoe Community College"
    },
    {
        "code": "0006",
        "value_represented": "Aurora East"
    },
    {
        "code": "0007",
        "value_represented": "Centennial Airport"
    },
    {
        "code": "0600",
        "value_represented": "SPECIAL STUDY CO SITE 1986/1987 LOCATED AT MILLER SCHOOL"
    },
    {
        "code": "0824",
        "value_represented": "SPECIAL STUDY CO SITE 1982 LOCATED AT MILLER SCHOOL (CITY OF ENGLWD)"
    },
    {
        "code": "0825",
        "value_represented": "GREENWOOD VILLAGE"
    },
    {
        "code": "1001",
        "value_represented": null
    },
   


The above response gives us a list of monitoring stations. Each monitoring station has a unique "code" which is a string number, and, sometimes, a description. The description seems to be something about where the monitoring station is located.


The above code response a list of 14 monitoring stations for Centennial, CO. I decided that this was sufficient data to proceed with my analysis.

## Example 3. Making a daily summary request

The function below is designed to encapsulate requests to the EPA AQS API. When calling the function one should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call simply pass in the parameters that need to change, like date ranges.

This function below is designed to request a daily summary of air quality data from the EPA for specified sensors over a defined date range. Users can provide parameters either through a dictionary or directly as function arguments. The function constructs an API request based on these parameters, ensuring all required fields are populated, and handles potential errors during the request process. It returns the JSON response containing the requested data or None if an error occurs.

This function streamlines the process of querying the AQS API for daily air quality summaries, making it easier to access historical data from 1964 to 2024. Furthermore, another filter is applied to only retrieve the data during "Wildfire Season" which is from May to October every year. 

Like mentioned previously, the API does have some rate limitations. For example, when I tried to get data for 60 years at once (instead of via a for loop), I ran into the error below. Hence, I decided to (1) limit my dates to only the wildfire months (2) run a loop over 60 years to overcome this error. 

```
Response for the gaseous pollutants ...
Data for 60 years ...
{
    "Header": [
        {
            "status": "Failed",
            "request_time": "2024-10-26T23:09:03.035-04:00",
            "url": "https://aqs.epa.gov/data/api/dailyData/byCounty?email=needula@uw.edu&key=bluemallard93&param=42101,42401,42602,44201&bdate=19640101&edate=20241026&state=08&county=005",
            "error": [
                "bdate: 19640101, edate: 20241026, only 1 year of data is permitted."
            ]
        }
    ]
}
```

In [58]:
# This function implements a daily summary request for air quality data from the EPA.
# It retrieves a daily summary value for each specified sensor from a start date to an end date.
# The function can accept parameters either through a dictionary or as individual arguments,
# with function parameters taking precedence over the request template parameters.

START_MMDD = "0501" 
END_MMDD = "1031"
YEAR_START = 1964
YEAR_END = 2024

def request_daily_summary(email_address=None, key=None, param=None,
                          begin_date=None, end_date=None, fips=None,
                          endpoint_url=API_REQUEST_URL, 
                          endpoint_action=API_ACTION_DAILY_SUMMARY_COUNTY, 
                          request_template=AQS_REQUEST_TEMPLATE,
                          headers=None):
    """
    Requests a daily summary of air quality data from the EPA.

    Parameters:
        email_address (str): The email address for API access.
        key (str): The API key for authentication.
        param (str): The specific parameter to retrieve (e.g., pollutant type).
        begin_date (str): The start date for the data request (format: YYYY-MM-DD).
        end_date (str): The end date for the data request (format: YYYY-MM-DD).
        fips (str): The FIPS code for the specific region (state and county).
        endpoint_url (str): The base URL for the API request.
        endpoint_action (str): The specific action to perform on the API.
        request_template (dict): A template for the API request parameters.
        headers (dict): Any additional headers to include in the request.

    Returns:
        dict: The JSON response containing the daily summary data.
    
    Raises:
        Exception: If required parameters are not provided.
    """
    
    # Update request_template with provided parameters, prioritizing function arguments over template defaults
    request_template.update({
        'email': email_address if email_address else request_template.get('email'),
        'key': key if key else request_template.get('key'),
        'param': param if param else request_template.get('param'),
        'begin_date': begin_date if begin_date else request_template.get('begin_date'),
        'end_date': end_date if end_date else request_template.get('end_date'),
    })
    
    # Handle FIPS code if provided
    if fips and len(fips) == 5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Ensure all required fields are populated
    for field in ['email', 'key', 'param', 'begin_date', 'end_date']:
        if not request_template[field]:
            raise Exception(f"Must supply {field} to call 'request_daily_summary()'")

    # Compose the request URL
    request_url = f"{endpoint_url}{endpoint_action.format(**request_template)}"
    
    # Make the request while handling potential errors
    try:
        # Throttle the request if necessary to avoid exceeding API limits
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        json_response = response.json()  # Parse JSON response
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        json_response = None
        
    return json_response

Hence, the function `extract_AQI_data` was created. The `extract_AQI_data` function is designed to process the JSON response received from an air quality index (AQI) API request. It checks the status of the response to determine whether the request was successful. If the status indicates "Success," the function extracts and returns the relevant AQI data, which consists of a list of dictionaries, each representing daily measurements for various sensors. If the status begins with "No data," the function does nothing and simply exits, implying that the request may need further inspection. In the event of any other status, it prints an error message along with the full JSON response for debugging purposes. If the function encounters any issues, it returns `None` to indicate that no valid data could be extracted. This function is integral to handling the response from the AQI API, allowing users to retrieve meaningful air quality data while managing potential errors effectively. 

Following the function definition, the code initializes a dictionary, `request_data`, using a template that includes the user's email and API key, as well as state and county information specific to Centennial, CO.

In [59]:
#
## Defining Date Constants
#

START_MMDD = "0501" 
END_MMDD = "1031"
YEAR_START = 1964
YEAR_END = 2024

def extract_AQI_data(aqi_JSON):
    """
    Extracts AQI Data from JSON response if the request is successful.
    
    Parameters: 
        aqi_JSON (dict): The raw JSON response of the request.
    Return:
        list: A list of dictionaries denoting each sensor's measurement for a day.
    """
    # Check if the request status is "Success"
    if aqi_JSON["Header"][0]['status'] == "Success":
        return aqi_JSON['Data']
    
    # Check if there's no data available
    if aqi_JSON["Header"][0]['status'].startswith("No data "):
        return []  # Return an empty list if there's no data
    
    # Handle any unexpected status
    print("Unexpected error occured!")
    print(json.dumps(aqi_JSON, indent=4))
    return None

# Prepare the request data by copying the template
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['centennial']['fips'][:2] 
request_data['county'] = CITY_LOCATIONS['centennial']['fips'][2:] 

No gaseous data available for 1964.
No gaseous data available for 1965.
No gaseous data available for 1966.
No gaseous data available for 1967.
No gaseous data available for 1968.
No gaseous data available for 1969.
No gaseous data available for 1970.
No gaseous data available for 1971.
No gaseous data available for 1972.
No gaseous data available for 1973.
No gaseous data available for 1974.
No gaseous data available for 1975.
No gaseous data available for 1976.
No gaseous data available for 1977.
No gaseous data available for 2008.


In [ ]:
%%time
gaseous_responses = []

# Iterate through all the years
for year in range(YEAR_START, YEAR_END + 1):
    # Define start and end times for the fire season
    begin_date = f"{year}{START_MMDD}"  # Formatted date string for start
    end_date = f"{year}{END_MMDD}"  # Formatted date string for end
    
    # Request daily summary data for gaseous pollutants
    request_data['param'] = AQI_PARAMS_GASEOUS
    gaseous_aqi = request_daily_summary(
        request_template=request_data, begin_date=begin_date, end_date=end_date)
    
    # Extract data from the response
    gas_list = extract_AQI_data(gaseous_aqi)

    # Append responses to the larger set if available
    if gas_list:  # If there is data, add it to the responses
        gaseous_responses.extend(gas_list)
    else:  # If no data is found for the year, notify
        print(f"No gaseous data available for {year}.")


In [76]:
with open("gaseous_aqi_temp_data", 'w') as json_file:
    json.dump(gaseous_responses, json_file, indent=4)  # indent for pretty printing

In [63]:
%%time
particulate_responses = []

for year in range(YEAR_START, YEAR_END + 1):
    # Define start and end times for the fire season
    begin_date = str(year) + START_MMDD
    end_date = str(year) + END_MMDD
    
        
    # Request daily summary data for particulate pollutants
    request_data['param'] = AQI_PARAMS_PARTICULATES
    particulate_aqi = request_daily_summary(
        request_template=request_data, begin_date=begin_date, end_date=end_date)
    part_list = extract_AQI_data(particulate_aqi)
    
        
    if part_list:
        particulate_responses.extend(part_list)
    else:
        print(f"No particulate data available for {year}.")

No particulate data available for 1964.
No particulate data available for 1965.
No particulate data available for 1966.
No particulate data available for 1967.
No particulate data available for 1968.
No particulate data available for 1969.
No particulate data available for 1970.
No particulate data available for 1971.
No particulate data available for 1972.
No particulate data available for 1973.
No particulate data available for 1974.
No particulate data available for 1975.
No particulate data available for 1976.
No particulate data available for 1977.
No particulate data available for 1978.
No particulate data available for 1979.
No particulate data available for 1980.
No particulate data available for 1981.
No particulate data available for 1982.
No particulate data available for 1983.
No particulate data available for 1984.
No particulate data available for 1985.
No particulate data available for 1995.
No particulate data available for 1996.
No particulate data available for 1997.


In [99]:
with open("particulate_aqi_temp_full", 'w') as json_file:
    json.dump(particulate_responses, json_file, indent=4)  # indent for pretty printing

{'state_code': '08',
 'county_code': '005',
 'site_number': '0005',
 'parameter_code': '88101',
 'poc': 1,
 'latitude': 39.604399,
 'longitude': -105.019526,
 'datum': 'WGS84',
 'parameter': 'PM2.5 - Local Conditions',
 'sample_duration_code': '7',
 'sample_duration': '24 HOUR',
 'pollutant_standard': 'PM25 24-hour 2006',
 'date_local': '2002-09-08',
 'units_of_measure': 'Micrograms/cubic meter (LC)',
 'event_type': 'No Events',
 'observation_count': 1,
 'observation_percent': 100.0,
 'validity_indicator': 'Y',
 'arithmetic_mean': 5.6,
 'first_max_value': 5.6,
 'first_max_hour': 0,
 'aqi': 31,
 'method_code': '118',
 'method': 'R & P Model 2025 PM2.5 Sequential w/WINS - GRAVIMETRIC',
 'local_site_name': 'Arapahoe Community College',
 'site_address': '6190 S. SANTA FE DR.',
 'state': 'Colorado',
 'county': 'Arapahoe',
 'city': 'Littleton',
 'cbsa_code': '19740',
 'cbsa': 'Denver-Aurora-Lakewood, CO',
 'date_of_last_change': '2024-09-12'}

In [156]:
df_particulate = pd.DataFrame(particulate_responses)
df_gaseous = pd.DataFrame(gaseous_responses)

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,08,005,1001,81102,1,39.628321,-104.988316,WGS84,PM10 Total 0-10um STP,7,...,062,HI-VOL-WEDDING-INLET - GRAVIMETRIC,None,4857 S BROADWAY,Colorado,Arapahoe,Englewood,19740,"Denver-Aurora-Lakewood, CO",2024-05-22
1,08,005,1001,81102,1,39.628321,-104.988316,WGS84,PM10 Total 0-10um STP,7,...,062,HI-VOL-WEDDING-INLET - GRAVIMETRIC,None,4857 S BROADWAY,Colorado,Arapahoe,Englewood,19740,"Denver-Aurora-Lakewood, CO",2024-05-22
2,08,005,1001,81102,1,39.628321,-104.988316,WGS84,PM10 Total 0-10um STP,7,...,062,HI-VOL-WEDDING-INLET - GRAVIMETRIC,None,4857 S BROADWAY,Colorado,Arapahoe,Englewood,19740,"Denver-Aurora-Lakewood, CO",2024-05-22
3,08,005,1001,81102,1,39.628321,-104.988316,WGS84,PM10 Total 0-10um STP,7,...,062,HI-VOL-WEDDING-INLET - GRAVIMETRIC,None,4857 S BROADWAY,Colorado,Arapahoe,Englewood,19740,"Denver-Aurora-Lakewood, CO",2024-05-22
4,08,005,1001,81102,1,39.628321,-104.988316,WGS84,PM10 Total 0-10um STP,7,...,062,HI-VOL-WEDDING-INLET - GRAVIMETRIC,None,4857 S BROADWAY,Colorado,Arapahoe,Englewood,19740,"Denver-Aurora-Lakewood, CO",2024-05-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13122,08,005,0005,88101,3,39.604399,-105.019526,WGS84,PM2.5 - Local Conditions,X,...,195,GRIMM EDM Model 180 with naphion dryer - Laser...,Arapahoe Community College,6190 S. SANTA FE DR.,Colorado,Arapahoe,Littleton,19740,"Denver-Aurora-Lakewood, CO",2024-10-17
13123,08,005,0005,88101,3,39.604399,-105.019526,WGS84,PM2.5 - Local Conditions,X,...,195,GRIMM EDM Model 180 with naphion dryer - Laser...,Arapahoe Community College,6190 S. SANTA FE DR.,Colorado,Arapahoe,Littleton,19740,"Denver-Aurora-Lakewood, CO",2024-10-17
13124,08,005,0005,88101,3,39.604399,-105.019526,WGS84,PM2.5 - Local Conditions,X,...,195,GRIMM EDM Model 180 with naphion dryer - Laser...,Arapahoe Community College,6190 S. SANTA FE DR.,Colorado,Arapahoe,Littleton,19740,"Denver-Aurora-Lakewood, CO",2024-10-17
13125,08,005,0005,88101,3,39.604399,-105.019526,WGS84,PM2.5 - Local Conditions,X,...,195,GRIMM EDM Model 180 with naphion dryer - Laser...,Arapahoe Community College,6190 S. SANTA FE DR.,Colorado,Arapahoe,Littleton,19740,"Denver-Aurora-Lakewood, CO",2024-10-17


In [157]:
df_particulate.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration_code',
       'sample_duration', 'pollutant_standard', 'date_local',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code', 'method',
       'local_site_name', 'site_address', 'state', 'county', 'city',
       'cbsa_code', 'cbsa', 'date_of_last_change'],
      dtype='object')

In [158]:
df_gaseous.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration_code',
       'sample_duration', 'pollutant_standard', 'date_local',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code', 'method',
       'local_site_name', 'site_address', 'state', 'county', 'city',
       'cbsa_code', 'cbsa', 'date_of_last_change'],
      dtype='object')

In [159]:
# Keep important columns
df_gaseous_select = df_gaseous[['state_code', 'county_code', 'site_number', 'parameter_code', 'latitude', 'longitude', 'parameter', 'sample_duration', 'date_local', 'units_of_measure', 'arithmetic_mean',
       'first_max_value', 'aqi']]
df_particulate_select = df_particulate[['state_code', 'county_code', 'site_number', 'parameter_code', 'latitude', 'longitude', 'parameter', 'sample_duration', 'date_local', 'units_of_measure', 'arithmetic_mean',
       'first_max_value', 'aqi']]

In [161]:
print(len(df_gaseous_select))
df_gaseous_unique = df_gaseous_select.drop_duplicates()
print(len(df_gaseous_unique))

print(len(df_particulate_select))
df_particulate_unique = df_particulate_select.drop_duplicates()
print(len(df_particulate_unique))

69406
53602
13127
1954


In [162]:
# Same format!
df_concat = pd.concat([df_gaseous_unique, df_particulate_unique], axis=0)
len(df_concat)
df_concat

55556

,state_code,county_code,site_number,parameter_code,latitude,longitude,parameter,sample_duration,date_local,units_of_measure,arithmetic_mean,first_max_value,aqi
0,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-16,Parts per million,0.000000,0.0,NaN
1,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,8-HR RUN AVG END HOUR,1978-09-16,Parts per million,0.000000,0.0,0.0
2,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-17,Parts per million,0.250000,1.0,NaN
3,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,8-HR RUN AVG END HOUR,1978-09-17,Parts per million,0.183333,0.4,5.0
4,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-18,Parts per million,0.625000,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13101,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,24-HR BLK AVG,2024-08-29,Micrograms/cubic meter (LC),5.400000,5.4,30.0
13109,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,1 HOUR,2024-08-30,Micrograms/cubic meter (LC),5.237500,8.0,NaN
13110,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,24-HR BLK AVG,2024-08-30,Micrograms/cubic meter (LC),5.200000,5.2,29.0
13118,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,1 HOUR,2024-08-31,Micrograms/cubic meter (LC),7.695833,13.5,NaN


## What's in my Data?

In [164]:
# How many sites exits? Good!
unique_combinations_of_stations = df_concat[['state_code', 'county_code', 'site_number']].drop_duplicates()
unique_combinations_of_stations

,state_code,county_code,site_number
0,08,005,0002
2522,08,005,1002
6313,08,005,0824
6347,08,005,0825
16447,08,005,0600
21522,08,005,0003
21634,08,005,0004
48834,08,005,0006
0,08,005,1001
241,08,005,0005


In [166]:
# All the pollutants
df_concat['parameter'].unique()

array(['Carbon monoxide', 'Sulfur dioxide', 'Ozone',
       'Nitrogen dioxide (NO2)', 'PM10 Total 0-10um STP',
       'PM2.5 - Local Conditions'], dtype=object)

## Why Haversine?

- smaller distances
- other one took too long
- reliable enough


In [169]:

# Reference coordinates
city_latitude = 39.59
city_longitude = -104.87

# Haversine function to calculate distance in miles
def haversine(lat1, lon1, lat2, lon2):
    R = 3956  # Earth radius in miles
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Calculate distances and add to DataFrame
df_concat['distance'] = df_concat.apply(lambda row: haversine(city_latitude, city_longitude, row['latitude'], row['longitude']), axis=1)

# Normalize distances for weighting (inverse weighting)
max_distance = df_concat['distance'].max()
min_distance = df_concat['distance'].min()

# print range
print(max_distance)
print(min_distance)

# Display the DataFrame with distances and weights
df_concat


16.339305037391007
1.9966097390437276


,state_code,county_code,site_number,parameter_code,latitude,longitude,parameter,sample_duration,date_local,units_of_measure,arithmetic_mean,first_max_value,aqi,distance,distance_weight
0,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-16,Parts per million,0.000000,0.0,NaN,4.884837,0.798627
1,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,8-HR RUN AVG END HOUR,1978-09-16,Parts per million,0.000000,0.0,0.0,4.884837,0.798627
2,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-17,Parts per million,0.250000,1.0,NaN,4.884837,0.798627
3,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,8-HR RUN AVG END HOUR,1978-09-17,Parts per million,0.183333,0.4,5.0,4.884837,0.798627
4,08,005,0002,42101,39.567887,-104.957193,Carbon monoxide,1 HOUR,1978-09-18,Parts per million,0.625000,1.0,NaN,4.884837,0.798627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13101,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,24-HR BLK AVG,2024-08-29,Micrograms/cubic meter (LC),5.400000,5.4,30.0,8.017025,0.580245
13109,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,1 HOUR,2024-08-30,Micrograms/cubic meter (LC),5.237500,8.0,NaN,8.017025,0.580245
13110,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,24-HR BLK AVG,2024-08-30,Micrograms/cubic meter (LC),5.200000,5.2,29.0,8.017025,0.580245
13118,08,005,0005,88101,39.604399,-105.019526,PM2.5 - Local Conditions,1 HOUR,2024-08-31,Micrograms/cubic meter (LC),7.695833,13.5,NaN,8.017025,0.580245


Since, everything is within a 20 mile radius, I will use the average of all sites

In [170]:
df_concat['date_local'].drop_duplicates()

0        1978-09-16
2        1978-09-17
4        1978-09-18
6        1978-09-19
8        1978-09-20
            ...    
13082    2024-08-27
13091    2024-08-28
13100    2024-08-29
13109    2024-08-30
13118    2024-08-31
Name: date_local, Length: 8378, dtype: object

In [171]:
df_concat['aqi'].isna().sum()

19335

In [172]:
# About 34% of the data is missing AQI!
df_concat['aqi'].isna().sum()/len(df_concat)

0.34802721578227375

In [173]:
# About 34% of the data is missing AQI!
df_concat['arithmetic_mean'].isna().sum()/len(df_concat)

0.0

In [174]:
# About 34% of the data is missing AQI!
df_concat['first_max_value'].isna().sum()/len(df_concat)

0.0

In [ ]:
# Let's use AQI if it exists or use the below formula to fill in null

In [143]:
# import pandas as pd
# import numpy as np

# def calculate_aqi_from_df_when_missing(df):
#     """
#     Calculate daily AQI values from a DataFrame containing air quality measurements,
#     weighted by distance from city center.
    
#     References:
#     1. EPA AQI Technical Assistance Document (2016)
#     """
    
#     # [Previous EPA_BREAKPOINTS and PARAMETER_MAPPING remain the same]
#     EPA_BREAKPOINTS = {
#         'CO': [  # Carbon Monoxide (8-hour)
#             (0.0, 4.4, 0, 50),
#             (4.5, 9.4, 51, 100),
#             (9.5, 12.4, 101, 150),
#             (12.5, 15.4, 151, 200),
#             (15.5, 30.4, 201, 300),
#             (30.5, 50.4, 301, 500)
#         ],
#         'SO2': [  # Sulfur Dioxide (1-hour)
#             (0, 35, 0, 50),
#             (36, 75, 51, 100),
#             (76, 185, 101, 150),
#             (186, 304, 151, 200),
#             (305, 604, 201, 300),
#             (605, 1004, 301, 500)
#         ],
#         'NO2': [  # Nitrogen Dioxide (1-hour)
#             (0, 53, 0, 50),
#             (54, 100, 51, 100),
#             (101, 360, 101, 150),
#             (361, 649, 151, 200),
#             (650, 1249, 201, 300),
#             (1250, 2049, 301, 500)
#         ],
#         'O3': [  # Ozone (8-hour)
#             (0.000, 0.054, 0, 50),
#             (0.055, 0.070, 51, 100),
#             (0.071, 0.085, 101, 150),
#             (0.086, 0.105, 151, 200),
#             (0.106, 0.200, 201, 300),
#             (0.201, 0.504, 301, 500)
#         ],
#         'PM2.5': [  # PM2.5 (24-hour)
#             (0.0, 12.0, 0, 50),
#             (12.1, 35.4, 51, 100),
#             (35.5, 55.4, 101, 150),
#             (55.5, 150.4, 151, 200),
#             (150.5, 250.4, 201, 300),
#             (250.5, 500.4, 301, 500)
#         ],
#         'PM10': [  # PM10 (24-hour)
#             (0, 54, 0, 50),
#             (55, 154, 51, 100),
#             (155, 254, 101, 150),
#             (255, 354, 151, 200),
#             (355, 424, 201, 300),
#             (425, 604, 301, 500)
#         ]
#     }
    
#     PARAMETER_MAPPING = {
#         '42101': 'CO',
#         '42401': 'SO2',
#         '42602': 'NO2',
#         '44201': 'O3',
#         '88101': 'PM2.5',
#         '81102': 'PM10'
#     }
    
#     def calculate_individual_aqi(concentration, pollutant):
#         """Calculate AQI for a single measurement"""
#         if pd.isna(concentration):
#             return np.nan
            
#         breakpoints = EPA_BREAKPOINTS[pollutant]
        
#         for Clow, Chigh, Ilow, Ihigh in breakpoints:
#             if Clow <= concentration <= Chigh:
#                 return round(((Ihigh - Ilow) / (Chigh - Clow)) * (concentration - Clow) + Ilow)
        
#         return np.nan

#     # Create a copy of the dataframe
#     df_processed = df.copy()
    
#     # Convert parameter_code to parameter names
#     df_processed['parameter'] = df_processed['parameter_code'].astype(str).map(PARAMETER_MAPPING)
    
#     # Use arithmetic_mean if available, otherwise first_max_value
#     df_processed['concentration'] = df_processed['arithmetic_mean'].fillna(df_processed['first_max_value'])
    
#     # Calculate individual AQI values
#     df_processed['calculated_aqi'] = df_processed.apply(
#         lambda row: calculate_individual_aqi(row['concentration'], row['parameter'])
#         if row['parameter'] in EPA_BREAKPOINTS else np.nan,
#         axis=1
#     )
    
#     # Apply distance weighting to the AQI values
#     df_processed['weighted_aqi'] = df_processed['calculated_aqi'] * df_processed['distance_weight']
    
#     # First, get the maximum weighted AQI for each date
#     daily_max = df_processed.groupby('date_local').agg({
#         'weighted_aqi': 'max',
#         'calculated_aqi': 'max'  # Keep original AQI for reference
#     }).reset_index()
#     daily_max = daily_max.rename(columns={'weighted_aqi': 'max_weighted_aqi', 'calculated_aqi': 'max_unweighted_aqi'})
    
#     # Merge back to get the parameters that correspond to the max weighted AQI
#     df_with_max = pd.merge(df_processed, daily_max, on='date_local')
    
#     # Get rows where weighted AQI equals the max weighted AQI for that day
#     max_aqi_rows = df_with_max[df_with_max['weighted_aqi'] == df_with_max['max_weighted_aqi']]
    
#     # Get the first parameter for each day (in case of ties)
#     daily_parameters = max_aqi_rows.groupby('date_local').agg({
#         'parameter': '
#         ',
#         'distance_weight': 'first'  # Keep the weight for reference
#     }).reset_index()
    
#     # Combine the results
#     daily_aqi = pd.merge(daily_max, daily_parameters, on='date_local')
#     daily_aqi = daily_aqi.rename(columns={
#         'max_weighted_aqi': 'weighted_aqi',
#         'max_unweighted_aqi': 'original_aqi',
#         'parameter': 'dominant_parameter'
#     })
    
#     # Add AQI category (based on weighted AQI)
#     def get_aqi_category(aqi):
#         if pd.isna(aqi):
#             return 'Unknown'
#         elif aqi <= 50:
#             return 'Good'
#         elif aqi <= 100:
#             return 'Moderate'
#         elif aqi <= 150:
#             return 'Unhealthy for Sensitive Groups'
#         elif aqi <= 200:
#             return 'Unhealthy'
#         elif aqi <= 300:
#             return 'Very Unhealthy'
#         else:
#             return 'Hazardous'
    
#     daily_aqi['aqi_category'] = daily_aqi['weighted_aqi'].apply(get_aqi_category)
    
#     # Reorder columns for clarity
#     daily_aqi = daily_aqi[[
#         'date_local',
#         'weighted_aqi',
#         'original_aqi',
#         'dominant_parameter',
#         'distance_weight',
#         'aqi_category'
#     ]]
    
#     return daily_aqi

# # Example usage:
# daily_aqi_results = calculate_aqi_from_df(df_concat)

In [175]:
def calculate_aqi_from_df_when_missing(df):
    """
    Calculate daily AQI values from a DataFrame containing air quality measurements.
    """
    
    EPA_BREAKPOINTS = {
        'CO': [  # Carbon Monoxide (8-hour)
            (0.0, 4.4, 0, 50),
            (4.5, 9.4, 51, 100),
            (9.5, 12.4, 101, 150),
            (12.5, 15.4, 151, 200),
            (15.5, 30.4, 201, 300),
            (30.5, 50.4, 301, 500)
        ],
        'SO2': [  # Sulfur Dioxide (1-hour)
            (0, 35, 0, 50),
            (36, 75, 51, 100),
            (76, 185, 101, 150),
            (186, 304, 151, 200),
            (305, 604, 201, 300),
            (605, 1004, 301, 500)
        ],
        'NO2': [  # Nitrogen Dioxide (1-hour)
            (0, 53, 0, 50),
            (54, 100, 51, 100),
            (101, 360, 101, 150),
            (361, 649, 151, 200),
            (650, 1249, 201, 300),
            (1250, 2049, 301, 500)
        ],
        'O3': [  # Ozone (8-hour)
            (0.000, 0.054, 0, 50),
            (0.055, 0.070, 51, 100),
            (0.071, 0.085, 101, 150),
            (0.086, 0.105, 151, 200),
            (0.106, 0.200, 201, 300),
            (0.201, 0.504, 301, 500)
        ],
        'PM2.5': [  # PM2.5 (24-hour)
            (0.0, 12.0, 0, 50),
            (12.1, 35.4, 51, 100),
            (35.5, 55.4, 101, 150),
            (55.5, 150.4, 151, 200),
            (150.5, 250.4, 201, 300),
            (250.5, 500.4, 301, 500)
        ],
        'PM10': [  # PM10 (24-hour)
            (0, 54, 0, 50),
            (55, 154, 51, 100),
            (155, 254, 101, 150),
            (255, 354, 151, 200),
            (355, 424, 201, 300),
            (425, 604, 301, 500)
        ]
    }
    
    PARAMETER_MAPPING = {
        '42101': 'CO',
        '42401': 'SO2',
        '42602': 'NO2',
        '44201': 'O3',
        '88101': 'PM2.5',
        '81102': 'PM10'
    }
    
    def calculate_individual_aqi(concentration, pollutant):
        """Calculate AQI for a single measurement"""
        if pd.isna(concentration):
            return np.nan
            
        breakpoints = EPA_BREAKPOINTS[pollutant]
        
        for Clow, Chigh, Ilow, Ihigh in breakpoints:
            if Clow <= concentration <= Chigh:
                return round(((Ihigh - Ilow) / (Chigh - Clow)) * (concentration - Clow) + Ilow)
        
        return np.nan

    # Create a copy of the dataframe
    df_processed = df.copy()
    
    # Convert parameter_code to parameter names
    df_processed['parameter'] = df_processed['parameter_code'].astype(str).map(PARAMETER_MAPPING)
    
    # Use the existing AQI if available, otherwise calculate using the function
    df_processed['aqi'] = df_processed['aqi'].fillna(
        df_processed.apply(
            lambda row: calculate_individual_aqi(row['arithmetic_mean'], row['parameter'])
            if row['parameter'] in EPA_BREAKPOINTS else np.nan,
            axis=1
        )
    )
    
    # Group by date_local and take the maximum AQI for each date
    daily_aqi = df_processed.groupby('date_local')['aqi'].max().reset_index()

    # Create a new dataframe to average AQI across each year
    daily_aqi['year'] = pd.to_datetime(daily_aqi['date_local']).dt.year
    yearly_aqi = daily_aqi.groupby('year')['aqi'].mean().reset_index()
    yearly_aqi.rename(columns={'aqi': 'average_aqi'}, inplace=True)
    
    return daily_aqi, yearly_aqi

# Example usage:
daily_aqi_results, yearly_aqi_results = calculate_aqi_from_df_when_missing(df_concat)


In [176]:
daily_aqi_results

,date_local,aqi,year
0,1978-06-21,49.0,1978
1,1978-06-22,90.0,1978
2,1978-06-23,54.0,1978
3,1978-06-24,108.0,1978
4,1978-06-25,44.0,1978
...,...,...,...
8373,2024-08-27,22.0,2024
8374,2024-08-28,21.0,2024
8375,2024-08-29,30.0,2024
8376,2024-08-30,29.0,2024


In [177]:
yearly_aqi_results

,year,average_aqi
0,1978,55.601504
1,1979,67.342391
2,1980,49.010929
3,1981,64.347826
4,1982,70.184783
5,1983,66.304348
6,1984,62.538043
7,1985,55.614130
8,1986,62.603261
9,1987,71.684783


In [178]:
yearly_aqi_results.to_csv("aqi_yearly.csv")

In [152]:
len(daily_aqi_results['date_local'].unique())

8374

# Step 2: 

In [54]:
# request_data['param'] = AQI_PARAMS_PARTICULATES
# # request daily summary data for the month of July in 2021
# particulate_aqi = request_daily_summary(request_template=request_data, begin_date="19640101", end_date="20241026")
# print("Response for the particulate pollutants ...")
# print("Data for 60 years ...")
# #
# if particulate_aqi["Header"][0]['status'] == "Success":
#     print(json.dumps(particulate_aqi['Data'],indent=4))
# elif particulate_aqi["Header"][0]['status'].startswith("No data "):
#     print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
# else:
#     print(json.dumps(particulate_aqi,indent=4))

Response for the particulate pollutants ...
Data for 60 years ...
{
    "Header": [
        {
            "status": "Failed",
            "request_time": "2024-10-26T23:09:24.994-04:00",
            "url": "https://aqs.epa.gov/data/api/dailyData/byCounty?email=needula@uw.edu&key=bluemallard93&param=81102,88101,88502&bdate=19640101&edate=20241026&state=08&county=005",
            "error": [
                "bdate: 19640101, edate: 20241026, only 1 year of data is permitted."
            ]
        }
    ]
}


In [55]:
# import json
# with open("particulate_aqi_temp_full", 'w') as json_file:
#     json.dump(particulate_aqi, json_file, indent=4)  # indent for pretty printing

The form of the daily summary response is a bit verbose with lots of repeated values. What we'll do is create a data structure that relies on a hierarchical context to summarize the data.

The two responses (for Bend, OR) show that not every monitoring site produces values. As well, it looks like the monitoring sites only produce values for particulates and not for gaseous pollutants.

The next function takes the response and a set of fields that should be extracted for their data values. The code assumes those fields are available. If there are missing values something could certainly go wrong. The function creates a summary for each monitoring site.

In [31]:
# #
# #    This is a list of field names - data - that will be extracted from each record
# #
# EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']

# #
# #    The function creates a summary record
# def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
#     ## the result will be structured around monitoring site, parameter, and then date
#     result = dict()
#     data = r["Data"]
#     for record in data:
#         # make sure the record is set up
#         site = record['site_number']
#         param = record['parameter_code']
#         #date = record['date_local']    # this version keeps the respnse value YYYY-
#         date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
#         if site not in result:
#             result[site] = dict()
#             result[site]['local_site_name'] = record['local_site_name']
#             result[site]['site_address'] = record['site_address']
#             result[site]['state'] = record['state']
#             result[site]['county'] = record['county']
#             result[site]['city'] = record['city']
#             result[site]['pollutant_type'] = dict()
#         if param not in result[site]['pollutant_type']:
#             result[site]['pollutant_type'][param] = dict()
#             result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
#             result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
#             result[site]['pollutant_type'][param]['method'] = record['method']
#             result[site]['pollutant_type'][param]['data'] = dict()
#         if date not in result[site]['pollutant_type'][param]['data']:
#             result[site]['pollutant_type'][param]['data'][date] = list()
        
#         # now extract the specified fields
#         extract = dict()
#         for k in fields:
#             if str(k) in record:
#                 extract[str(k)] = record[k]
#             else:
#                 # this makes sure we always have the requested fields, even if
#                 # we have a missing value for a given day/month
#                 extract[str(k)] = None
        
#         # add this extraction to the list for the day
#         result[site]['pollutant_type'][param]['data'][date].append(extract)
    
#     return result


In [33]:

# extract_gaseous = extract_summary_from_response(gaseous_aqi)
# print("Summary of gaseous extraction ...")
# print(json.dumps(extract_gaseous,indent=4))


Summary of gaseous extraction ...
{
    "0002": {
        "local_site_name": "HIGHLAND RESERVOIR",
        "site_address": "8100 S. UNIVERSITY BLVD",
        "state": "Colorado",
        "county": "Arapahoe",
        "city": "Southglenn",
        "pollutant_type": {
            "44201": {
                "parameter_name": "Ozone",
                "units_of_measure": "Parts per million",
                "method": "INSTRUMENTAL - ULTRA VIOLET ABSORPTION",
                "data": {
                    "20210701": [
                        {
                            "sample_duration": "1 HOUR",
                            "observation_count": 24,
                            "arithmetic_mean": 0.040833,
                            "aqi": null
                        },
                        {
                            "sample_duration": "8-HR RUN AVG BEGIN HOUR",
                            "observation_count": 24,
                            "arithmetic_mean": 0.039083,
            

In [34]:

# extract_particulate = extract_summary_from_response(particulate_aqi)
# print("Summary of particulate extraction ...")
# print(json.dumps(extract_particulate,indent=4))


Summary of particulate extraction ...
{
    "0005": {
        "local_site_name": "Arapahoe Community College",
        "site_address": "6190 S. SANTA FE DR.",
        "state": "Colorado",
        "county": "Arapahoe",
        "city": "Littleton",
        "pollutant_type": {
            "88101": {
                "parameter_name": "PM2.5 - Local Conditions",
                "units_of_measure": "Micrograms/cubic meter (LC)",
                "method": "R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric",
                "data": {
                    "20210718": [
                        {
                            "sample_duration": "24 HOUR",
                            "observation_count": 1,
                            "arithmetic_mean": 9.0,
                            "aqi": 50
                        },
                        {
                            "sample_duration": "24 HOUR",
                            "observation_count": 1,
                           

## Example 4. Making request by bounding box

There are some places that don't have monitoring stations. In the EPA FAQ that covers the AQS system, they note that their monitoring covers 2000 of the 3000+ US counties.

The AQS API has a mechanism of requesting data and monitoring stations using a geographic bounding box. The above examples just demonstrated the use of the AQS API for making requests by counties. The examples below illustrate the use of bounding boxes. The example below makes requests to identify monitoring stations within the bounding box. Once you knew you have monitoring stations, then the bounding box could be used in the daily summary requests to get AQS data.


In [30]:
# #
# #   These are rough estimates for creating bounding boxes based on a city location
# #   You can find these rough estimates on the USGS website:
# #   https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps
# #
# LAT_25MILES = 25.0 * (1.0/69.0)    # This is about 25 miles of latitude in decimal degrees
# LON_25MILES = 25.0 * (1.0/54.6)    # This is about 25 miles of longitude in decimal degrees
# #
# #   Compute rough estimates for a bounding box around a given place
# #   The bounding box is scaled in 50 mile increments. That is, the bounding box will have sides that
# #   are rough multiples of 50 miles, with the center of the box around the indicated place.
# #   The scale parameter determines the scale (size) of the bounding box
# #
# def bounding_latlon(place=None,scale=1.0):
#     minlat = place['latlon'][0] - float(scale) * LAT_25MILES
#     maxlat = place['latlon'][0] + float(scale) * LAT_25MILES
#     minlon = place['latlon'][1] - float(scale) * LON_25MILES
#     maxlon = place['latlon'][1] + float(scale) * LON_25MILES
#     return [minlat,maxlat,minlon,maxlon]



In [28]:
# #
# #    This implements the monitors request. This requests monitoring stations. This can be done by state, county, or bounding box. 
# #
# #    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
# #    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
# #
# def request_monitors(email_address = None, key = None, param=None,
#                           begin_date = None, end_date = None, fips = None,
#                           endpoint_url = API_REQUEST_URL, 
#                           endpoint_action = API_ACTION_MONITORS_COUNTY, 
#                           request_template = AQS_REQUEST_TEMPLATE,
#                           headers = None):
    
#     #  This prioritizes the info from the call parameters - not what's already in the template
#     if email_address:
#         request_template['email'] = email_address
#     if key:
#         request_template['key'] = key
#     if param:
#         request_template['param'] = param
#     if begin_date:
#         request_template['begin_date'] = begin_date
#     if end_date:
#         request_template['end_date'] = end_date
#     if fips and len(fips)==5:
#         request_template['state'] = fips[:2]
#         request_template['county'] = fips[2:]            

#     # Make sure there are values that allow us to make a call - these are always required
#     if not request_template['email']:
#         raise Exception("Must supply an email address to call 'request_monitors()'")
#     if not request_template['key']: 
#         raise Exception("Must supply a key to call 'request_monitors()'")
#     if not request_template['param']: 
#         raise Exception("Must supply param values to call 'request_monitors()'")
#     if not request_template['begin_date']: 
#         raise Exception("Must supply a begin_date to call 'request_monitors()'")
#     if not request_template['end_date']: 
#         raise Exception("Must supply an end_date to call 'request_monitors()'")
#     # Note we're not validating FIPS fields because not all of the monitors actions require the FIPS numbers
    
#     # compose the request
#     request_url = endpoint_url+endpoint_action.format(**request_template)
    
#     # make the request
#     try:
#         # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
#         # during the request processing - throttling is always a good practice with a free data source
#         if API_THROTTLE_WAIT > 0.0:
#             time.sleep(API_THROTTLE_WAIT)
#         response = requests.get(request_url, headers=headers)
#         json_response = response.json()
#     except Exception as e:
#         print(e)
#         json_response = None
#     return json_response


In [29]:
# #
# request_data = AQS_REQUEST_TEMPLATE.copy()
# request_data['email'] = USERNAME
# request_data['key'] = APIKEY
# request_data['param'] = AQI_PARAMS_PARTICULATES     # remember we have both gaseous and particulates
# # 
# #   We got the monitoring stations for Bend OR above (Deschutes county) - let's work with that one again
# request_data['state'] = CITY_LOCATIONS['centennial']['fips'][:2]
# request_data['county'] = CITY_LOCATIONS['centennial']['fips'][2:]
# #
# # the first example uses the default - request monitors by county, we'll just use a recent date for now
# response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731")
# #
# # the response should be similar to the 'list' request above - but in this case we should only get monitors that
# # monitor the AQI_PARAMS_PARTICULATES set of params.
# #
# if response["Header"][0]['status'] == "Success":
#     print(json.dumps(response['Data'],indent=4))
# else:
#     print(json.dumps(response,indent=4))


[
    {
        "state_code": "08",
        "county_code": "005",
        "site_number": "0005",
        "parameter_code": "88101",
        "poc": 1,
        "parameter_name": "PM2.5 - Local Conditions",
        "open_date": "1999-03-10",
        "close_date": "2024-04-01",
        "concurred_exclusions": null,
        "dominant_source": null,
        "measurement_scale": "NEIGHBORHOOD",
        "measurement_scale_def": "500 M TO 4KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "145",
        "last_method_description": "R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric",
        "last_method_begin_date": "2003-06-05",
        "naaqs_primary_monitor": null,
        "qa_primary_monitor": null,
        "monitor_type": null,
        "networks": null,
        "monitoring_agency_code": "0240",
        "monitoring_agency": "Colorado Department of Public Health And Environment",
        "si_id": 1680,
        "latitude": 39.604399,
    


Given the response above for Bend, OR - it looks like there are 5 monitoring sites that are in Deschutes County - AND that monitor particulate air quality. Their unique site IDs are given in the 'site_number' field of the dictionary. The response also includes their lat,lon positions should we need that. Comparing the site numbers to the response from the list request above - all of the sites here are there. So, things seem to be heading in the right direction.



Next, we're going to change the monitors request to use a bounding box and see what happens.


In [20]:
# #
# #    Create a copy of the AQS_REQUEST_TEMPLATE
# #
# request_data = AQS_REQUEST_TEMPLATE.copy()
# request_data['email'] = USERNAME
# request_data['key'] = APIKEY
# request_data['param'] = AQI_PARAMS_PARTICULATES     # same particulate request as the one abover
# # 
# #   Not going to use these - comment them out
# #request_data['state'] = CITY_LOCATIONS['bend']['fips'][:2]
# #request_data['county'] = CITY_LOCATIONS['bend']['fips'][2:]
# #
# #   Now, we need bounding box parameters

# #   50 mile box
# bbox = bounding_latlon(CITY_LOCATIONS['centennial'],scale=1.0)
# #   100 mile box
# #bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=2.0)
# #   150 mile box
# #bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=3.0)
# #   200 mile box
# #bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=4.0)

# # the bbox response comes back as a list - [minlat,maxlat,minlon,maxlon]

# #   put our bounding box into the request_data
# request_data['minlat'] = bbox[0]
# request_data['maxlat'] = bbox[1]
# request_data['minlon'] = bbox[2]
# request_data['maxlon'] = bbox[3]

# #
# #   we need to change the action for the API from the default to the bounding box - same recent date for now
# response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731",
#                             endpoint_action = API_ACTION_MONITORS_BOX)
# #
# #
# #
# if response["Header"][0]['status'] == "Success":
#     print(json.dumps(response['Data'],indent=4))
# else:
#     print(json.dumps(response,indent=4))


[
    {
        "state_code": "41",
        "county_code": "017",
        "site_number": "0120",
        "parameter_code": "88101",
        "poc": 1,
        "parameter_name": "PM2.5 - Local Conditions",
        "open_date": "2001-03-01",
        "close_date": null,
        "concurred_exclusions": null,
        "dominant_source": "AREA",
        "measurement_scale": "NEIGHBORHOOD",
        "measurement_scale_def": "500 M TO 4KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "209",
        "last_method_description": "Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM2.5C - Beta Attenuation",
        "last_method_begin_date": "2023-09-30",
        "naaqs_primary_monitor": "Y",
        "qa_primary_monitor": null,
        "monitor_type": "SPM",
        "networks": null,
        "monitoring_agency_code": "0821",
        "monitoring_agency": "Oregon Department Of Environmental Quality",
        "si_id": 13113,
        "latitude": 44.063916,
        "longitud


The response above indicates that all five of the particulate monitoring stations in Deschutes County are within (roughly) 25 miles of Bend, OR. Our 50 mile bounding box was centered on Bend - and we got all 5 stations.

There are places where monitoring stations might be hard to find. This would mean that air quality data is harder to get for that location.

Lets, consider Seaside, OR.


In [21]:
# #
# request_data = AQS_REQUEST_TEMPLATE.copy()
# request_data['email'] = USERNAME
# request_data['key'] = APIKEY
# request_data['param'] = AQI_PARAMS_PARTICULATES     # like the above example, just focusing on particulates
# # 
# #    This time, we'll try getting the list of monitoring stations for Seaside, OR
# request_data['state'] = CITY_LOCATIONS['centennial']['fips'][:2]
# request_data['county'] = CITY_LOCATIONS['centennial']['fips'][2:]
# #
# #    For the Seaside example, let's get everything that we can find in the county.
# response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731")
# #
# #
# if response["Header"][0]['status'] == "Success":
#     print(json.dumps(response['Data'],indent=4))
# elif response["Header"][0]['status'].startswith("No data "):
#     print("Looks like the response generated no monitoring stations.")
# else:
#     print(json.dumps(response,indent=4))


Looks like the response generated no monitoring stations.



The response indicates that there are no monitoring stations in Clatsop County (where Seaside, OR is located). The EPA says they only have monitoring stations in about 2000 of the 3000+ US counties. If we're going to find monitoring stations - maybe we can use the bounding box to find them.

Let's scale up some bounding boxes and see if we can find some monitoring stations for Seaside OR.


In [22]:

# request_data = AQS_REQUEST_TEMPLATE.copy()
# request_data['email'] = USERNAME
# request_data['key'] = APIKEY
# request_data['param'] = AQI_PARAMS_PARTICULATES     # same particulate request as the one abover
# # 
# #   Not going to use these - comment them out
# #request_data['state'] = CITY_LOCATIONS['seaside']['fips'][:2]
# #request_data['county'] = CITY_LOCATIONS['seaside']['fips'][2:]
# #
# #   Now, we need bounding box parameters

# #   50 mile box - roughly within 25 miles of the location
# #bbox = bounding_latlon(CITY_LOCATIONS['seaside'],scale=1.0)
# #   100 mile box - roughly within 50 miles of the location
# bbox = bounding_latlon(CITY_LOCATIONS['centennial'],scale=2.0)
# #   150 mile box - roughly within 75 miles of the location
# #bbox = bounding_latlon(CITY_LOCATIONS['seaside'],scale=3.0)
# #   200 mile box
# #bbox = bounding_latlon(CITY_LOCATIONS['seaside'],scale=4.0)

# # the bbox response comes back as a list - [minlat,maxlat,minlon,maxlon]

# #   put our bounding box into the request_data
# request_data['minlat'] = bbox[0]
# request_data['maxlat'] = bbox[1]
# request_data['minlon'] = bbox[2]
# request_data['maxlon'] = bbox[3]

# #
# #   we need to change the action for the API from the default to the bounding box - same recent date for now
# response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731",
#                             endpoint_action = API_ACTION_MONITORS_BOX)
# #
# #
# #
# if response["Header"][0]['status'] == "Success":
#     print(json.dumps(response['Data'],indent=4))
# else:
#     print(json.dumps(response,indent=4))


[
    {
        "state_code": "41",
        "county_code": "067",
        "site_number": "0006",
        "parameter_code": "88502",
        "poc": 3,
        "parameter_name": "Acceptable PM2.5 AQI & Speciation Mass",
        "open_date": "2019-06-20",
        "close_date": null,
        "concurred_exclusions": null,
        "dominant_source": "AREA",
        "measurement_scale": "NEIGHBORHOOD",
        "measurement_scale_def": "500 M TO 4KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "791",
        "last_method_description": "OTHR AUTOMATD 2.5 MASS CONCENT - SURROGATE MEASURE",
        "last_method_begin_date": "2019-06-20",
        "naaqs_primary_monitor": null,
        "qa_primary_monitor": null,
        "monitor_type": "SPM",
        "networks": null,
        "monitoring_agency_code": "0821",
        "monitoring_agency": "Oregon Department Of Environmental Quality",
        "si_id": 104536,
        "latitude": 45.52483,
        "longitude": 


Looks like we can find two monitoring stations within 50 miles of Seaside, OR. If we needed to know the air quality in Seaside, we could use a Daily Summary request with the same bounding box and get data for these two stations - which are the closest ones we could find.
